In [6]:
import re
import os
import numpy as np
import pandas as pd


def result_file_name(args):
    file_name = f"results_{args.fold_range}_{args.seed}"
    if len(args.modalities) > 0:
        file_name += "_"
        file_name += "_".join(args.modalities)
    return file_name

class Args:
    def __init__(self, result_folder, modalities=["clinical", "miRNA", "mRNA", "WSI"]):
        self.modalities = modalities
        self.fold_range = 5
        self.seed = 24
        self.result_path = f"./{result_folder}/{result_file_name(self)}"

 
all_folders = ["single"]
all_folder_labels = ["Single Modality"]
all_modalities = np.array(["clinical", "miRNA", "mRNA", "WSI"])
all_results = []
for result_folder in all_folders:
    test_dict = {}
    exp_results = []
    exp_labels = []
    for i in range(1, 2 ** len(all_modalities)):
        selection = F"{i:0{len(all_modalities)}b}"
        modality_select = all_modalities[np.array(list(selection)) == '1']
        args = Args(result_folder, modality_select)
        if not os.path.exists(args.result_path):
            continue
        all_loss = []
        all_train_loss, all_train_c_index = [], []
        all_val_loss, all_val_c_index = [], []
        all_test_loss, all_test_c_index = [], []
        for k in range(args.fold_range):
            filename = f"{args.result_path}/result_fold{k}.log"
            if not os.path.exists(filename):
                print(f"Missing File: {filename}")
                break
            with open(filename) as file:
                lines = [line.rstrip() for line in file]
            for line in lines:
                if bool(re.search('Final Train', line)):
                    loss, c_index = map(float, re.findall("\d+\.\d+", line))
                    all_train_loss.append(loss)
                    all_train_c_index.append(c_index)
                if bool(re.search('Final Val', line)):
                    loss, c_index = map(float, re.findall("\d+\.\d+", line))
                    all_val_loss.append(loss)
                    all_val_c_index.append(c_index)
                if bool(re.search('Test', line)):
                    loss, c_index = map(float, re.findall("\d+\.\d+", line))
                    all_test_loss.append(loss)
                    all_test_c_index.append(c_index)
        train_loss = np.mean(all_train_loss)
        train_c_index = np.mean(all_train_c_index)

        val_loss = np.mean(all_val_loss)
        val_c_index = np.mean(all_val_c_index)

        test_loss = np.mean(all_test_loss)
        test_c_index = np.mean(all_test_c_index)
        test_std = np.std(all_test_c_index)

        exp_results.append(f"{test_c_index:.4f} ({test_std:.4f})")
        exp_labels.append("+".join(modality_select))
    all_results.append(exp_results)
all_results = np.array(all_results).T
pd.DataFrame(all_results, index=np.array(exp_labels), columns=all_folder_labels)

/home/sajjad/miniconda3/envs/int_env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/sajjad/miniconda3/envs/int_env/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Single Modality
WSI,0.7304 (0.0141)
mRNA,0.7502 (0.0313)
miRNA,0.7052 (0.0270)
clinical,0.5670 (0.0750)
